In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
books = pd.read_csv("D:\\Data Science-ExcelR\\Assignments\\Recommendation system\\book.csv",sep=',',encoding= 'unicode_escape')
books.head()

,Unnamed: 0,User.ID,Book.Title,Book.Rating
0,1,276726,Classical Mythology,5
1,2,276729,Clara Callan,3
2,3,276729,Decision in Normandy,6
3,4,276736,Flu: The Story of the Great Influenza Pandemic...,8
4,5,276737,The Mummies of Urumchi,6


In [25]:
books= books.rename(columns={ books.columns[0]: "Details"})
books= books.rename(columns={ books.columns[2]: "Book_Title"})

In [15]:
books

,Details,User.ID,Book.Title,Book.Rating
0,1,276726,Classical Mythology,5
1,2,276729,Clara Callan,3
2,3,276729,Decision in Normandy,6
3,4,276736,Flu: The Story of the Great Influenza Pandemic...,8
4,5,276737,The Mummies of Urumchi,6
...,...,...,...,...
9995,9996,162121,American Fried: Adventures of a Happy Eater.,7
9996,9997,162121,Cannibal In Manhattan,9
9997,9998,162121,How to Flirt: A Practical Guide,7
9998,9999,162121,Twilight,8


In [26]:
print("Number of rows: ", len(books))
print("Number of columns: ",len(books.columns))
print("Number of rows and columns: ",books.shape)
print("Number of elements: ",books.size)

Number of rows:  10000
Number of columns:  4
Number of rows and columns:  (10000, 4)
Number of elements:  40000


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = 'english')

In [28]:
books.columns.isnull()
books["Book_Title"].isnull()
books["Book_Title"] = books["Book_Title"].fillna(" ")

In [29]:
tfidf_matrix = tfidf.fit_transform(books.Book_Title)
print("Shape of Original DF : ",books.shape)
print("Shape of TFIDF Matrix: ",tfidf_matrix.shape)

Shape of Original DF :  (10000, 4)
Shape of TFIDF Matrix:  (10000, 11435)


In [30]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim_matrix = linear_kernel(tfidf_matrix,tfidf_matrix) # cos_sim_matrix is obtained by matrix * matrix
print(cosine_sim_matrix)

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [33]:
books['Book_Title']= books['Book_Title'].str.lower()
books.Book_Title.value_counts()

stardust                                                                                      5
fahrenheit 451                                                                                5
vanished                                                                                      4
charlie and the chocolate factory                                                             4
the amber spyglass (his dark materials, book 3)                                               4
                                                                                             ..
don quijote de la mancha                                                                      1
carry on jeeves                                                                               1
confessions of saint augustine (image book)                                                   1
when someone you love has a mental illness: a handbook for family, friends, and caregivers    1
sins of the seventh sister              

In [34]:
books.drop_duplicates(subset ="Book_Title",keep='first',inplace = True)
books.reset_index(drop=True,inplace = True)

In [35]:
books_index = pd.Series(books.index, index = books['Book_Title'])
books_index.sort_values(ascending=True, inplace=True)
print(books_index)

Book_Title
classical mythology                                                                                      0
clara callan                                                                                             1
decision in normandy                                                                                     2
flu: the story of the great influenza pandemic of 1918 and the search for the virus that caused it       3
the mummies of urumchi                                                                                   4
                                                                                                      ... 
american fried: adventures of a happy eater.                                                          9633
cannibal in manhattan                                                                                 9634
how to flirt: a practical guide                                                                       9635
twilight                  

In [47]:
def get_book_recommendations(BookName,topN):
   
    #1:get Book index(id) using its title bcz user will enter movie name not index number
    books_id = books_index[BookName] 
    #print(book_id)
    
    #2: getting the pair wise similarity score of all the movies with that of enterted movie; here enumerate act as counter
    cosine_scores = list(enumerate(cosine_sim_matrix[books_id])) 
    #print(cosine_scores)
    
    #3: Sorted the obtained score; sort lamda x:x[1] means in dictionary x = [12293(x[0]movie id),0.0(x[1]cosine score)],we shall sort on cosine score
    cosine_scores = sorted(cosine_scores, key=lambda x:x[1], reverse = True)
    #print(cosine_scores)
    
    #4: slice the sorted list as per user input (topN); it can top5,top10 or anything
    cosine_score_TopN = cosine_scores[0:topN]
    print(cosine_score_TopN)

    #5:above will give only index number not the movie name. but customer requires movie name
    # Getting movie index from the top 10 sorted list X[0]= Movie id and X[1] is similarity score
    books_idx = [i[0] for i in cosine_score_TopN]
    books_scores = [i[1] for i in cosine_score_TopN]
    
    #6. Create dataframe to store the similar movie list
    books_similar_show = pd.DataFrame(columns=["Book_Title","Score"])
    
    
    #7. Put the topN data into dataframe
    books_similar_show["Book_Title"] = books.loc[books_idx,"Book_Title"] # using anime index we fetch movie name from anime
    books_similar_show["Score"]= books_scores
    
# #     print("Before Reset:",book_similar_show)
    books_similar_show.reset_index(inplace=True) # this will show anime_idx value as seperate columns
#     print("After Reset:",book_similar_show)
    books_similar_show.drop(["index"],axis=1,inplace=True)
    
    #8.OutPut
    return books_similar_show

In [48]:
Top_N_Books = get_book_recommendations(3,5)
Top_N_Books

[(3, 1.0000000000000002), (7760, 0.23030946840620475), (1341, 0.21897028740099653), (7552, 0.2046583436772717), (2376, 0.16639108548710613)]


,Book_Title,Score
0,flu: the story of the great influenza pandemic...,1.000000
1,el respeto hacia uno mismo,0.230309
2,men and brethren,0.218970
3,the golfer's book of wisdom: common sense and ...,0.204658
4,death of a river guide,0.166391


In [49]:
Top_N_Books = get_book_recommendations(3,7)
Top_N_Books

[(3, 1.0000000000000002), (7760, 0.23030946840620475), (1341, 0.21897028740099653), (7552, 0.2046583436772717), (2376, 0.16639108548710613), (4201, 0.1648553751164242), (9471, 0.16072806477544846)]


,Book_Title,Score
0,flu: the story of the great influenza pandemic...,1.000000
1,el respeto hacia uno mismo,0.230309
2,men and brethren,0.218970
3,the golfer's book of wisdom: common sense and ...,0.204658
4,death of a river guide,0.166391
5,"palestina, ocupacio i resistencia: manual prac...",0.164855
6,babies and other hazards of sex : how to make ...,0.160728
